In [2]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region_name = boto3.Session().region_name

In [3]:
#variable to store all tables created
ingest_create_athena_table_csv_passed = False

In [4]:
#Retrieving stores database
%store -r ingest_create_athena_db_passed

In [5]:
#Checking for returned database
print(ingest_create_athena_db_passed)

True


In [6]:
#Retrieving stored private s3 bucket
%store -r s3_private_path_csv_credit_card_trans
%store -r s3_private_path_csv_user_credit_card_trans
%store -r s3_private_path_csv_card_info
%store -r s3_private_path_csv_user_info

In [7]:
#Checking bucket is returned
print(s3_private_path_csv_credit_card_trans)
print(s3_private_path_csv_user_credit_card_trans)
print(s3_private_path_csv_card_info)
print(s3_private_path_csv_user_info)

s3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/credit_card_trans/
s3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/user_credit_card_trans/
s3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/card_info/
s3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/user_info/


In [8]:
#Creating tables in database
!pip install pyathena
import pandas as pd
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor
from pyathena.pandas.util import as_pandas

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [9]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_directory = "s3://{0}/athena/staging".format(bucket)

In [10]:
# Set athena database name and table names
database_name = "credit_card_fraud_db"
table_one_csv = "user_credit_card_trans"
table_two_csv = "credit_card_trans"
table_three_csv = "cards_info"
table_four_csv = "users_info"

In [11]:
cursor = connect(region_name = region_name, s3_staging_dir = s3_staging_directory).cursor()

In [12]:
#SQL statement to create table 1

table_one = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        `User` int,
        Card int,
        Year int,
        Month int,
        Day int,
        Time string,
        Amount string,
        Use_Chip string,
        Merchant_Name string,
        Merchant_City string,
        Merchant_State string,
        Zip double,
        MCC int,
        Errors string,
        Is_Fraud string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_one_csv, s3_private_path_csv_user_credit_card_trans
)
print(table_one)

CREATE EXTERNAL TABLE IF NOT EXISTS credit_card_fraud_db.user_credit_card_trans(
        `User` int,
        Card int,
        Year int,
        Month int,
        Day int,
        Time string,
        Amount string,
        Use_Chip string,
        Merchant_Name string,
        Merchant_City string,
        Merchant_State string,
        Zip double,
        MCC int,
        Errors string,
        Is_Fraud string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/user_credit_card_trans/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [13]:
cursor.execute(table_one)

In [14]:
#SQL statement to create table 2

table_two = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        `User` int,
        Card int,
        Year int,
        Month int,
        Day int,
        Time string,
        Amount string,
        Use_Chip string,
        Merchant_Name string,
        Merchant_City string,
        Merchant_State string,
        Zip double,
        MCC int,
        Errors string,
        Is_Fraud string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_two_csv, s3_private_path_csv_credit_card_trans
)
print(table_two)

CREATE EXTERNAL TABLE IF NOT EXISTS credit_card_fraud_db.credit_card_trans(
        `User` int,
        Card int,
        Year int,
        Month int,
        Day int,
        Time string,
        Amount string,
        Use_Chip string,
        Merchant_Name string,
        Merchant_City string,
        Merchant_State string,
        Zip double,
        MCC int,
        Errors string,
        Is_Fraud string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/credit_card_trans/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [15]:
cursor.execute(table_two)

In [16]:
#SQL statement to execute

table_three = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        `User` int,
        CARD_INDEX int,
        Card_Brand string,
        Card_Type string,
        Card_Number int,
        Expires string,
        CVV int,
        Has_Chip string,
        Cards_Issued int,
        Credit_Limit string,
        Acct_Open_Date string,
        Year_PIN_last_Changed int,
        Card_On_Dark_Web string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_three_csv, s3_private_path_csv_card_info
)
print(table_three)

CREATE EXTERNAL TABLE IF NOT EXISTS credit_card_fraud_db.cards_info(
        `User` int,
        CARD_INDEX int,
        Card_Brand string,
        Card_Type string,
        Card_Number int,
        Expires string,
        CVV int,
        Has_Chip string,
        Cards_Issued int,
        Credit_Limit string,
        Acct_Open_Date string,
        Year_PIN_last_Changed int,
        Card_On_Dark_Web string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/card_info/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [17]:
cursor.execute(table_three)

In [18]:
#SQL statement to execute

table_four = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
        Person string,
        Current_Age int,
        Retirement_Age int,
        Birth_Year int,
        Birth_Month int,
        Gender string,
        Address string,
        Apartment int,
        City string,
        State string,
        Zipcode int,
        Latitude double,
        Longitude double,
        Per_Capita_Income_Zipcode string,
        yearly_income_person string,
        Total_Debt string,
        Fico_score int,
        Num_Credit_Cards int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_four_csv, s3_private_path_csv_user_info
)
print(table_four)

CREATE EXTERNAL TABLE IF NOT EXISTS credit_card_fraud_db.users_info(
        Person string,
        Current_Age int,
        Retirement_Age int,
        Birth_Year int,
        Birth_Month int,
        Gender string,
        Address string,
        Apartment int,
        City string,
        State string,
        Zipcode int,
        Latitude double,
        Longitude double,
        Per_Capita_Income_Zipcode string,
        yearly_income_person string,
        Total_Debt string,
        Fico_score int,
        Num_Credit_Cards int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-584771966647/finalproject-datasets/csv/user_info/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [19]:
cursor.execute(table_four)

In [20]:
#Verifying tables are created
statement = "SHOW TABLES in {}".format(database_name)

cursor.execute(statement)

In [21]:
df_tables = as_pandas(cursor)
df_tables.head()

,tab_name
0,cards_info
1,credit_card_trans
2,user_credit_card_trans
3,users_info


In [22]:
#variable to store all joined tables
ingest_joined_athena_table_csv_passed = False

In [23]:
conn = connect(region_name = region_name, s3_staging_dir = s3_staging_directory)

# Querying with Athena

In [24]:
card_on_dark_web = 'Yes'
statement = """SELECT * FROM {}.{} 
            WHERE card_on_dark_web = '{}' LIMIT 1000""".format(database_name, table_three_csv, card_on_dark_web)

print(statement)

SELECT * FROM credit_card_fraud_db.cards_info 
            WHERE card_on_dark_web = 'Yes' LIMIT 1000


In [25]:
df_cards_info = pd.read_sql('SELECT * FROM {}.{} LIMIT 1000'.format(database_name, table_three_csv), conn)

In [26]:
df_cards_info.head()

,user,card_index,card_brand,card_type,card_number,expires,cvv,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,0,0,Visa,Debit,None,12/2022,623,YES,2,$24295,09/2002,2008,No
1,0,1,Visa,Debit,None,12/2020,393,YES,2,$21968,04/2014,2014,No
2,0,2,Visa,Debit,None,02/2024,719,YES,2,$46414,07/2003,2004,No
3,0,3,Visa,Credit,None,08/2024,693,NO,1,$12400,01/2003,2012,No
4,0,4,Mastercard,Debit (Prepaid),None,03/2009,75,YES,1,$28,09/2008,2009,No


In [27]:
df_user_info = pd.read_sql('SELECT * FROM {}.{} LIMIT 1000'.format(database_name, table_four_csv), conn)

In [28]:
df_user_info.head()

,person,current_age,retirement_age,birth_year,birth_month,gender,address,apartment,city,state,zipcode,latitude,longitude,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards
0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,NaN,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5
1,Sasha Sadr,53,68,1966,12,Female,3606 Federal Boulevard,NaN,Little Neck,NY,11363,40.76,-73.74,$37891,$77254,$191349,701,5
2,Saanvi Lee,81,67,1938,11,Female,766 Third Drive,NaN,West Covina,CA,91792,34.02,-117.89,$22681,$33483,$196,698,5
3,Everlee Clark,63,63,1957,1,Female,3 Madison Street,NaN,New York,NY,10069,40.71,-73.99,$163145,$249925,$202328,722,4
4,Kyle Peterson,43,70,1976,9,Male,9620 Valley Stream Drive,NaN,San Francisco,CA,94117,37.76,-122.44,$53797,$109687,$183855,675,1


In [29]:
df_credit_card_trans = pd.read_sql('SELECT * FROM {}.{} LIMIT 1000'.format(database_name, table_two_csv), conn)

In [30]:
df_credit_card_trans.head()

,user,card,year,month,day,time,amount,use_chip,merchant_name,merchant_city,merchant_state,zip,mcc,errors,is_fraud
0,1252,0,2011,7,14,20:04,$63.34,Swipe Transaction,-4716840478510425106,Purcell,OK,73080.0,7230,,No
1,1252,0,2011,7,14,21:38,$16.13,Swipe Transaction,5817218446178736267,Spencer,OK,73084.0,5912,,No
2,1252,0,2011,7,15,09:20,$46.87,Swipe Transaction,2463763636533349415,Rush Springs,OK,73082.0,7349,,No
3,1252,0,2011,7,16,20:12,$30.34,Swipe Transaction,-2472481739355111587,Wynnewood,OK,73098.0,7538,,No
4,1252,0,2011,7,18,16:38,$26.12,Swipe Transaction,-6913437630297895625,Purcell,OK,73080.0,5300,,No


In [31]:
#joining the three dataframes using pandas
df_cred_info = pd.merge(df_credit_card_trans, df_cards_info, how = 'inner', on='user')
df_cred_info.head()

,user,card,year,month,day,time,amount,use_chip,merchant_name,merchant_city,...,card_type,card_number,expires,cvv,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web


In [32]:
df_joined_tables = pd.merge(df_cred_info, df_user_info, how ='inner', left_on = 'merchant_city', right_on = 'city')
df_joined_tables.head()

,user,card,year,month,day,time,amount,use_chip,merchant_name,merchant_city,...,city,state,zipcode,latitude,longitude,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards


In [33]:
if table_one_csv and table_two_csv and table_three_csv and table_four_csv in df_tables.values:
    ingest_create_athena_table_csv_passed = True

In [34]:
#storing all variables
%store ingest_create_athena_table_csv_passed

Stored 'ingest_create_athena_table_csv_passed' (bool)


In [35]:
if not df_tables.empty:
    print("[OK]")
else:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA. LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


In [36]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                         -> True
ingest_create_athena_table_csv_passed                  -> True
ingest_exploration_passed                              -> False
s3_private_path_csv_card_info                          -> 's3://sagemaker-us-east-1-584771966647/finalprojec
s3_private_path_csv_credit_card_trans                  -> 's3://sagemaker-us-east-1-584771966647/finalprojec
s3_private_path_csv_user_credit_card_trans             -> 's3://sagemaker-us-east-1-584771966647/finalprojec
s3_private_path_csv_user_info                          -> 's3://sagemaker-us-east-1-584771966647/finalprojec
s3_public_path_csv_card_info                           -> 's3://finalproject-datasets/csv/card_info/'
s3_public_path_csv_credit_card_trans                   -> 's3://finalproject-datasets/csv/credit_card_trans/
s3_public_path_csv_user_credit_card_trans              -> 's3://finalproject-datasets/csv/user_credit_card_t
s3_public_pat

In [37]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [38]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>